In [ ]:
!pip install tensorflow


In [ ]:
!unzip -q /content/fruits.zip -d /content/fruits


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import os


In [ ]:
#Prepare the Dataset

# Paths to the train and test directories
train_dir = "/content/fruits/fruits/train"
test_dir = "/content/fruits/fruits/test"

# Data generators for preprocessing
train_datagen = ImageDataGenerator(rescale=1.0/255, horizontal_flip=True, zoom_range=0.2)
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load the train and test data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 294 images belonging to 2 classes.
Found 97 images belonging to 2 classes.


In [ ]:
#Load pretrained model

# Load MobileNetV2 without the top layers
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model layers

# Add custom layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)  # Number of classes in your dataset

model = Model(inputs=base_model.input, outputs=predictions)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
#Compile the model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model

history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10  
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.6195 - loss: 0.7044 - val_accuracy: 0.8351 - val_loss: 0.4010
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.9349 - loss: 0.2203 - val_accuracy: 0.8454 - val_loss: 0.3468
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.9604 - loss: 0.1605 - val_accuracy: 0.8660 - val_loss: 0.3201
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.9632 - loss: 0.1166 - val_accuracy: 0.8557 - val_loss: 0.3044
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - accuracy: 0.9796 - loss: 0.0667 - val_accuracy: 0.8247 - val_loss: 0.3498
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.9630 - loss: 0.0888 - val_accuracy: 0.8144 - val_loss: 0.3908
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.9887 - loss: 0.0486 - val_accuracy: 0.8247 - val_loss: 0.3917
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.9972 - loss: 0.0377 - val_accuracy: 0.8351 - val_loss: 0.3794
Epo

In [ ]:
#Evaluate the Model

loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 844ms/step - accuracy: 0.8724 - loss: 0.3320
Test Accuracy: 86.60%


In [ ]:
#Save the model

model.save("apple_tomato_classifier_model.h5")


In [ ]:
model.save("apple_tomato_classifier_model.keras")


In [ ]:
!ls

apple_tomato_classifier_model.h5     fruits	 sample_data
apple_tomato_classifier_model.keras  fruits.zip


In [ ]:
#Test with random images

from tensorflow.keras.preprocessing import image
import numpy as np

# Load an image
img_path = "/content/fruits/fruits/test/apples/img_p1_133.jpeg"   # check the apple
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predict
predictions = model.predict(img_array)
predicted_class = train_generator.class_indices
print(f"Predicted Class: {list(predicted_class.keys())[np.argmax(predictions)]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Predicted Class: apples


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load an image
img_path = "/content/fruits/fruits/test/tomatoes/img_p3_126.jpeg"   # check the tomato
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predict
predictions = model.predict(img_array)
predicted_class = train_generator.class_indices
print(f"Predicted Class: {list(predicted_class.keys())[np.argmax(predictions)]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Predicted Class: tomatoes
